## Questão 01

In [6]:
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [7]:
response_vectors = np.array([
    [1, -1, -1, -1, -1, -1, -1, -1],
    [-1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, 1, -1, -1, -1, -1, -1],
    [-1, -1, -1, 1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 1, -1, -1, -1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 1, -1],
    [-1, -1, -1, -1, -1, -1, -1, 1]
], dtype=np.float32)

data_X = []
data_y = []

for i in range(10000):
    instance = np.array([random.randint(0,1), random.randint(0,1), random.randint(0,1)])
    label = instance[0]*4 + instance[1]*2 + instance[2]
    noise = np.array([random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1)])

    data_X.append(instance+noise)
    data_y.append(label)

data_X = np.array(data_X, dtype=np.float32)
data_y = np.array(data_y, dtype=np.float32)

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=10)

In [8]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

clf = Perceptron(tol=1e-3, random_state=10, validation_fraction=0.2, early_stopping=True, verbose=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

-- Epoch 1
Norm: 3.40, NNZs: 3, Bias: 1.000000, T: 6400, Avg. loss: 0.000171
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 3.40, NNZs: 3, Bias: 1.000000, T: 12800, Avg. loss: 0.000000
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 3.40, NNZs: 3, Bias: 1.000000, T: 19200, Avg. loss: 0.000000
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 3.40, NNZs: 3, Bias: 1.000000, T: 25600, Avg. loss: 0.000000
Total training time: 0.02 seconds.
-- Epoch 5
Norm: 3.40, NNZs: 3, Bias: 1.000000, T: 32000, Avg. loss: 0.000000
Total training time: 0.02 seconds.
-- Epoch 6
Norm: 3.40, NNZs: 3, Bias: 1.000000, T: 38400, Avg. loss: 0.000000
Total training time: 0.02 seconds.
Convergence after 6 epochs took 0.02 seconds
-- Epoch 1
Norm: 4.18, NNZs: 3, Bias: -1.000000, T: 6400, Avg. loss: 0.001184
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 4.18, NNZs: 3, Bias: -1.000000, T: 12800, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 4.18, NNZs: 3, Bias: -1.000000, T:

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


In [9]:
## plotar gráfico tridimensional com os pontos e suas respectivas classes usando o plotly
import plotly.graph_objects as go
fig = go.Figure(data=[go.Scatter3d(x=X_test[:,0], y=X_test[:,1], z=X_test[:,2], mode='markers', marker=dict(size=3, color=y_pred))])
fig.show()

## Questão 02

### a)

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

n = 10000
data_X = []
data_y = []

for i in range(n):
    instance = np.array([random.uniform(-5,5), random.uniform(-5,5)])

    if instance[0]*instance[1] > 0:
        label = 1
    elif instance[0]*instance[1] < 0:
        label = 0
    else:
        continue

    data_X.append(instance)
    data_y.append(label)


data_X = np.array(data_X)
data_y = torch.tensor(data_y, dtype=torch.float32).reshape(-1,1)

In [33]:
# Dividindo os dados em conjuntos de treinamento, validação e teste
inputs_train, inputs_test, targets_train, targets_test = train_test_split(data_X, data_y,
                                                                          test_size=0.2,random_state=42)

# dividindo 80% dos dados de treino para treino e 20% para validação
inputs_train, inputs_val, targets_train, targets_val = train_test_split(inputs_train, targets_train,
                                                                        test_size=0.25, random_state=42)

# Convertendo para tensores do PyTorch
inputs_train = torch.tensor(inputs_train, dtype=torch.float32)
targets_train = torch.tensor(targets_train, dtype=torch.float32)
inputs_val = torch.tensor(inputs_val, dtype=torch.float32)
targets_val = torch.tensor(targets_val, dtype=torch.float32)
inputs_test = torch.tensor(inputs_test, dtype=torch.float32)
targets_test = torch.tensor(targets_test, dtype=torch.float32)

# Definindo o modelo MLP
class XOR_MLP(nn.Module):
    def __init__(self):
        super(XOR_MLP, self).__init__()
        self.fc1 = nn.Linear(2, 4)  # camada de entrada para 4 neurônios
        self.fc2 = nn.Linear(4, 1)  # camada oculta para 1 neurônio de saída

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Instanciando o modelo
model = XOR_MLP()

# Definindo a função de perda e o otimizador
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Treinamento do modelo
epochs = 1000
for epoch in range(epochs):
    # Forward pass
    outputs = model(inputs_train)
    loss = criterion(outputs, targets_train)

    # Backward pass e otimização
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Avaliação do modelo no conjunto de validação
with torch.no_grad():
    outputs_val = model(inputs_val)
    predicted_val = torch.round(outputs_val)
    accuracy_val = (predicted_val == targets_val).sum().item() / predicted_val.size(0)
    print(f'Acurácia do modelo no conjunto de validação: {accuracy_val:.2f}')

# Avaliação final do modelo no conjunto de teste
with torch.no_grad():
    outputs_test = model(inputs_test)
    predicted_test = torch.round(outputs_test)
    accuracy_test = (predicted_test == targets_test).sum().item() / predicted_test.size(0)
    print(f'Acurácia do modelo no conjunto de teste: {accuracy_test:.2f}')


C:\Users\jahon\AppData\Local\Temp\ipykernel_6856\4086301176.py:7: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\jahon\AppData\Local\Temp\ipykernel_6856\4086301176.py:9: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\jahon\AppData\Local\Temp\ipykernel_6856\4086301176.py:11: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch [100/1000], Loss: 0.1816
Epoch [200/1000], Loss: 0.0890
Epoch [300/1000], Loss: 0.0615
Epoch [400/1000], Loss: 0.0493
Epoch [500/1000], Loss: 0.0421
Epoch [600/1000], Loss: 0.0373
Epoch [700/1000], Loss: 0.0338
Epoch [800/1000], Loss: 0.0311
Epoch [900/1000], Loss: 0.0290
Epoch [1000/1000], Loss: 0.0272
Acurácia do modelo no conjunto de validação: 0.99
Acurácia do modelo no conjunto de teste: 1.00
